In [17]:
#!pip install imutils

In [18]:
#!pip install cmake

In [19]:
#!pip install dlib

In [20]:
from scipy.spatial import distance as dist
from imutils import face_utils
import imutils
import numpy as np
import time
import dlib
import cv2
import os
import glob

def smile(mouth):
    A = dist.euclidean(mouth[3], mouth[9])
    B = dist.euclidean(mouth[2], mouth[10])
    C = dist.euclidean(mouth[4], mouth[8])
    avg = (A+B+C)/3
    D = dist.euclidean(mouth[0], mouth[6])
    mar=avg/D
    return mar

ModuleNotFoundError: No module named 'imutils'

In [ ]:
#MAR selfie application

counter = 0
selfie_no = 0

shape_predictor= "shape_predictor_68_face_landmarks.dat" 
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(shape_predictor)


(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

print("[INFO] starting video stream thread...")
vs = cv2.VideoCapture(0)


while True:
    ret, frame = vs.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)
    for rect in rects:
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        mouth= shape[mStart:mEnd]
        mar= smile(mouth)
        mouthHull = cv2.convexHull(mouth)
        cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)
       

        if mar < .25 or mar > .38:
            cv2.putText(frame,"Smile Detected",(10, 60), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 3, cv2.LINE_AA) 
            counter = counter + 1
            
            if counter > 20 :
                cv2.putText(frame,"3",(10, 90), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 3, cv2.LINE_AA)
                
            if counter > 40 :
                cv2.putText(frame,"2",(30, 90), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 3, cv2.LINE_AA)
                
            if counter > 60 :
                cv2.putText(frame,"1",(50, 90), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 3, cv2.LINE_AA)
            
            if counter > 80 :
                ret,frame1 = vs.read()     
                cv2.putText(frame,"Captured",(10, 120), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255), 3, cv2.LINE_AA)
                
            if counter == 90 :
                selfie_no = selfie_no+1
                img_name = "images/mar_webcam/mar_smart_selfie_{}.png".format(selfie_no)
                time.sleep(2)
                cv2.imwrite(img_name,frame1)
                counter = 0
                
        else :
            counter = 0

        cv2.putText(frame, "MAR: {}".format(mar), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    cv2.imshow("Frame", frame)

    key2 = cv2.waitKey(1) & 0xFF
    if key2 == ord('q'):
        break


cv2.destroyAllWindows()

print("[INFO] video stream ended..")

In [ ]:
#split frames from video
 
# Opens the Video file
cap = cv2.VideoCapture('dataset/videos/Video Test.mp4')
i = 0

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    i = i + 1
    img_name = "dataset/videos/videos_actual_2/frame{}.png".format(i)
    cv2.imwrite(img_name,frame)

cap.release()
cv2.destroyAllWindows()

In [ ]:
#test video MAR

def MAR_video(file):
    smile_no = 0

    shape_predictor= "shape_predictor_68_face_landmarks.dat" 
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(shape_predictor)


    (mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

    print("[INFO] starting video...")
    vs = cv2.VideoCapture(file)

    while(vs.isOpened()):
        ret, frame = vs.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)
        smile_no = smile_no + 1
        frame1 = frame.copy()
              
        for rect in rects:   
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)
            mouth= shape[mStart:mEnd]
            mar= smile(mouth)
            mouthHull = cv2.convexHull(mouth)
            cv2.drawContours(frame, [mouthHull], -1, (0, 255, 0), 1)
            
            if (mar < .25 or mar > .38):
                cv2.putText(frame,"Smile Detected",(10, 60), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 3, cv2.LINE_AA)       
                cv2.putText(frame,"Captured",(10, 120), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255), 2)
                img_name = "images/video_mar/mar_video_test_{}.png".format(smile_no)
                cv2.imwrite(img_name,frame1)
                  
            cv2.putText(frame, "MAR: {}".format(mar), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            
        cv2.imshow("Video Testing", frame)
        

        key2 = cv2.waitKey(1) & 0xFF
        if key2 == ord('q'):
            break


    cv2.destroyAllWindows()

    print("[INFO] video ended..")
    

videofile = 'dataset/videos/Make People Smile Project.mp4'
MAR_video(videofile)

In [ ]:
#test image MAR

pos_dir = 'dataset/images/positives'
neg_dir = 'dataset/images/negatives'

def MAR_image(img, file):
    
    print("[INFO] Image " + file + "..")
    
    scale_percent = 200 # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    # resize image
    resized_img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

    shape_predictor= "shape_predictor_68_face_landmarks.dat" 
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(shape_predictor)

    (mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

    rects = detector(resized_img, 0)

    for rect in rects:
        print("mouth")
        shape = predictor(resized_img, rect)
        shape = face_utils.shape_to_np(shape)
        mouth= shape[mStart:mEnd]
        mar= smile(mouth)
        mouthHull = cv2.convexHull(mouth)
        cv2.drawContours(resized_img, [mouthHull], -1, (0, 255, 0), 1)

        if mar < .25 or mar > .38 :  
            file_split = file.split('\\')
            file_split1 = file_split[1].split('.')
            filename = 'images/positive_mar/' + file_split1[0] + '.png' #change directory
            cv2.imwrite(filename, resized_img)


#to call all image files from dataset 
data_path = os.path.join(pos_dir, '*g') #change positive/negative dataset
files = glob.glob(data_path)
data = [] 
for f1 in files: 
    image = cv2.imread(f1) 
    data.append(image)
    MAR_image(image, f1)

print("[INFO] End of images...")